In [278]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate

# 1. Load Data

In [279]:
df = pd.read_csv('data/compact_data.csv')
df

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,192.168.10.3-192.168.10.9-53-56719-17,192.168.10.9,56719.0,192.168.10.3,53.0,17.0,03/07/2017 11:12:04,46640.0,2.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,1,192.168.10.3-192.168.10.16-53-29457-17,192.168.10.16,29457.0,192.168.10.3,53.0,17.0,7/7/2017 10:02,62118.0,2.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,2,192.168.10.9-72.21.91.29-1662-80-6,72.21.91.29,80.0,192.168.10.9,1662.0,6.0,4/7/2017 9:07,117.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,3,192.168.10.1-192.168.10.3-53-61235-17,192.168.10.3,61235.0,192.168.10.1,53.0,17.0,5/7/2017 9:40,152866.0,1.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,4,192.168.10.17-194.244.5.206-43837-443-6,192.168.10.17,43837.0,194.244.5.206,443.0,6.0,03/07/2017 12:43:56,4.0,2.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54995,54995,172.16.0.1-192.168.10.50-33854-80-6,172.16.0.1,33854.0,192.168.10.50,80.0,6.0,5/7/2017 10:15,108750317.0,6.0,...,32.0,3007401.0,0.0,3007401.0,3007401.0,106000000.0,0.0,106000000.0,106000000.0,DoS Slowhttptest
54996,54996,172.16.0.1-192.168.10.50-38696-80-6,172.16.0.1,38696.0,192.168.10.50,80.0,6.0,5/7/2017 10:26,63085232.0,7.0,...,40.0,7005402.0,0.0,7005402.0,7005402.0,18700000.0,12200000.0,32000000.0,8015911.0,DoS Slowhttptest
54997,54997,172.16.0.1-192.168.10.50-37390-80-6,172.16.0.1,37390.0,192.168.10.50,80.0,6.0,5/7/2017 10:22,105213438.0,6.0,...,32.0,3004176.0,0.0,3004176.0,3004176.0,102000000.0,0.0,102000000.0,102000000.0,DoS Slowhttptest
54998,54998,172.16.0.1-192.168.10.50-49436-80-6,172.16.0.1,49436.0,192.168.10.50,80.0,6.0,5/7/2017 10:33,63137052.0,7.0,...,40.0,7009683.0,0.0,7009683.0,7009683.0,18700000.0,12200000.0,32100000.0,8015940.0,DoS Slowhttptest


# 2. Preprocessing

## Select X and Normalzation

In [280]:
x_label=[
       'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size',
       'Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk',
       'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
       'Idle Std', 'Idle Max', 'Idle Min']

df[x_label]=df[x_label].apply(
    lambda x: (x - x.min()) / (x.max()-x.min()))
df = df.fillna(0)

## Label Encoding

In [281]:
from sklearn.preprocessing import LabelEncoder 

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

In [282]:
x = df[x_label].to_numpy()
y = df['Label'].to_numpy()

In [283]:
print(x.shape)
print(y.shape)

(55000, 77)
(55000,)


## Train / Test

In [284]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.8, test_size = 0.2)

In [285]:
print(train_y.shape)
print(train_y)
pd.Series(train_y).value_counts()

(44000,)
[2 0 8 ... 8 0 0]


0    12010
1     4033
2     4015
4     4005
6     4004
3     3999
8     3991
5     3976
7     3967
dtype: int64

# 3. Modeling 1

## Logistic Regression

In [286]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=30, random_state=42)
lr.fit(train_x, train_y)

print('train:',lr.score(train_x, train_y))
print('test:',lr.score(test_x, test_y))

lr_train=lr.predict(train_x)
lr_test=lr.predict(test_x)

train: 0.8866136363636363
test: 0.885090909090909


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Decision Tree

In [287]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_x, train_y)

print('train:',dt.score(train_x, train_y))
print('test:',dt.score(test_x, test_y))

dt_train=dt.predict(train_x)
dt_test=dt.predict(test_x)

train: 0.9997272727272727
test: 0.9960909090909091


## Random Forest

In [288]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)

rf.fit(train_x, train_y)

print('train:',rf.score(train_x, train_y))
print('test:',rf.score(test_x, test_y))

rf_train=rf.predict(train_x)
rf_test=rf.predict(test_x)

train: 0.9991363636363636
test: 0.9972727272727273


## XGBoost

In [289]:
# xgboost
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
xgb.fit(train_x, train_y)

print('train:',xgb.score(train_x, train_y))
print('test:',xgb.score(test_x, test_y))

xgb_train=xgb.predict(train_x)
xgb_test=xgb.predict(test_x)

train: 0.9995
test: 0.9986363636363637


# 4. Feature Selection

In [290]:
dt_feature = dt.feature_importances_
rf_feature = rf.feature_importances_
xgb_feature = xgb.feature_importances_

features = (dt_feature + rf_feature + xgb_feature)/3
features_series = pd.Series(features, index=df[x_label].columns.values)

features_series_selected = features_series.nlargest(int(len(features)*0.20))
feature_selected = features_series_selected.index


print('feature count: %d to %d' %(len(features), len(features_series_selected)))
print(features_series_selected)

x_selected = df[feature_selected].to_numpy()

feature count: 77 to 15
Bwd Packet Length Min      0.101470
Init_Win_bytes_backward    0.075737
Fwd Packet Length Std      0.063725
Packet Length Variance     0.044884
Min Packet Length          0.040844
Flow IAT Mean              0.035842
Packet Length Mean         0.032258
PSH Flag Count             0.028936
Bwd Packet Length Std      0.027433
Init_Win_bytes_forward     0.025857
Average Packet Size        0.024974
Idle Min                   0.023864
Fwd PSH Flags              0.023527
Packet Length Std          0.023038
Active Min                 0.021165
dtype: float64


# 5. Modeling 2 - After Feature Selected

## Train / Test

In [291]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x_selected, y, train_size = 0.8, test_size = 0.2)

## Logistic Regression

In [292]:
from sklearn.linear_model import LogisticRegression

lr2 = LogisticRegression(C=30, random_state=42)
lr2.fit(train_x, train_y)

print('train:',lr2.score(train_x, train_y))
print('test:',lr2.score(test_x, test_y))

lr2_train=lr2.predict(train_x)
lr2_test=lr2.predict(test_x)

train: 0.7566136363636363
test: 0.761


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Decision Tree

In [293]:
from sklearn.tree import DecisionTreeClassifier
dt2 = DecisionTreeClassifier(random_state=42)
dt2.fit(train_x, train_y)

print('train:',dt2.score(train_x, train_y))
print('test:',dt2.score(test_x, test_y))

# Before:
# 0.9997272727272727
# 0.9957272727272727

dt2_train=dt2.predict(train_x)
dt2_test=dt2.predict(test_x)

train: 0.9995909090909091
test: 0.9954545454545455


## Random Foreset

In [294]:
from sklearn.ensemble import RandomForestClassifier

rf2 = RandomForestClassifier(n_jobs=-1, random_state=42)

rf2.fit(train_x, train_y)

print('train:',rf2.score(train_x, train_y))
print('test:',rf2.score(test_x, test_y))

# Beofre:
# 0.99925
# 0.9977272727272727

rf2_train=rf2.predict(train_x)
rf2_test=rf2.predict(test_x)

train: 0.9995227272727273
test: 0.9975454545454545


## XGBoost

In [295]:
from xgboost import XGBClassifier

xgb2 = XGBClassifier(tree_method='hist', random_state=42)
xgb2.fit(train_x, train_y)

print('train:',xgb2.score(train_x, train_y))
print('test:',xgb2.score(test_x, test_y))

# Before:
# 0.9995454545454545
# 0.9981818181818182

xgb2_train=xgb2.predict(train_x)
xgb2_test=xgb2.predict(test_x)

train: 0.9989318181818182
test: 0.9978181818181818


# 6. Ensemble

In [296]:
# not including Logsitc Regression
estimators = [
    ('DecisionTree', dt2),
    ('RandomForest', rf2),
    ('XgBoost', xgb2)
    ]

pd.DataFrame( {
    'DecisionTree': dt2_train,
    'RandomForest': rf2_train,
    'XgBoost': xgb2_train,
    })

,DecisionTree,RandomForest,XgBoost
0,4,4,4
1,0,0,0
2,4,4,4
3,1,1,1
4,0,0,0
...,...,...,...
43995,2,2,2
43996,2,2,2
43997,3,3,3
43998,5,5,5


In [297]:
from sklearn.ensemble import VotingClassifier

vt = VotingClassifier(estimators, voting='soft')
vt.fit(train_x, train_y)

print('train:',vt.score(train_x, train_y))
print('test:',vt.score(test_x, test_y))

vt_train=vt.predict(train_x)
vt_test=vt.predict(test_x)

train: 0.9995909090909091
test: 0.9978181818181818
